In [1]:
import pandas as pd
import re

# Baca daftar buku dari file CSV
book_list = pd.read_csv('assets/book_names.csv')['Kitab'].tolist()

# Buat pola regex untuk mencocokkan nama buku, nomor, dan tanda hubung
pattern = r'\b(?:' + '|'.join(book_list) + r')\b\s*\d*(?:-\d*)?'

# Fungsi untuk mengolah teks
def process_text(text):
    # Temukan semua kecocokan dengan pola regex
    matches = re.findall(pattern, text)

    # Gabungkan kecocokan menjadi satu string
    processed_text = ' '.join(matches)

    return processed_text

# Contoh penggunaan
text = "yoh 7-8slse"
processed_text = process_text(text)
print(processed_text)  # Output yang diharapkan: "yoh 7-8"


yoh 7-8
